# ML Zoomcamp - Midterm Project

For the ml-zoomcamp midterm project I've chosen to train a classifier model
that predicts the presence or absence of heart disease based on a series of attributes.

## Dataset : UCI Heart Disease Data

The dataset used is a subset of the Heart Disease Data Set from UCI Machine Learning data repository. It contains 14 patient attributes and I'll use them to predict whether a patient has heart disease (target values 1,2,3,4) or not (target value 0).

The dataset is included in the project directory, or can be downloaded from kaggle:

[https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data/download?datasetVersionNumber=6](https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data/download?datasetVersionNumber=6)

The feature names are:

0. **id**
1. **age**
2. **sex**
3. **dataset**: the Cleveland database is the only one used
4. **cp**: chest pain type
    - typical angina
    - atypical angina
    - non-anginal pain
    - asymptomatic
5. **trestbps**: resting blood pressure (in mm Hg on admission to the hospital)
6. **chol**: serum cholestoral in mg/dl
7. **fbs**: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
8. **restecg**: resting electrocardiographic results
    - normal
    - lv hypertrophy	
    - st-t abnormality
9. **thalach**: maximum heart rate achieved
10. **exang**: exercise induced angina
11. **oldpeak**: ST depression induced by exercise relative to rest
12. **slope**: the slope of the peak exercise ST segment
    - upsloping
    - flat
    - downsloping
13. **ca**: number of major vessels (0-3) colored by flourosopy
14. **thal**:
    - normal
    - fixed defect
    - reversable defect
15. **num**: the predicted target
    - normal = 0
    - heart disease = 1,2,3,4

## Data preparation and exploratory data analysis

In [383]:
import pandas as pd

After downloading the dataset, I imported it with pandas
to inspect the features and possible missing values.

In [384]:
df = pd.read_csv("../heart_disease_uci.csv")
df.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [385]:
df.columns

Index(['id', 'age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs',
       'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'],
      dtype='object')

Column names are all lower case and contain no spaces.

When checking for missing values:

In [386]:
df.isnull().sum()

id            0
age           0
sex           0
dataset       0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64

There are missing values in several columns, so I decide to impute NaNs in the columns with less than 50% of missing values.
- For categorical features, I manually find the most common value with `value_counts()`.
- For numerical features, I impute NaN values with the median.

I don't think it makes sense to keep columns where values are missing in more than 50% of the rows,
so I delete them from the dataframe.
The `id` column doesn't provide any useful information either, so I delete it too.

In [387]:
df["fbs"].fillna(False, inplace=True)
df["restecg"].fillna("normal", inplace=True)
df["exang"].fillna(False, inplace=True)
df["slope"].fillna("flat", inplace=True)

numerical_nans = ["trestbps", "chol", "thalch", "oldpeak"]
for col in numerical_nans:
    median = df[col].median()
    df[col].fillna(median, inplace=True)

del df["id"]
del df["ca"]
del df["thal"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       920 non-null    int64  
 1   sex       920 non-null    object 
 2   dataset   920 non-null    object 
 3   cp        920 non-null    object 
 4   trestbps  920 non-null    float64
 5   chol      920 non-null    float64
 6   fbs       920 non-null    bool   
 7   restecg   920 non-null    object 
 8   thalch    920 non-null    float64
 9   exang     920 non-null    bool   
 10  oldpeak   920 non-null    float64
 11  slope     920 non-null    object 
 12  num       920 non-null    int64  
dtypes: bool(2), float64(4), int64(2), object(5)
memory usage: 81.0+ KB


In [388]:
df.num.value_counts()

0    411
1    265
2    109
3    107
4     28
Name: num, dtype: int64

There are 5 unique values in the target column `num`: 0 indicates that the patient has no heart disease
and 1,2,3,4 indicate the presence of heart disease to various degrees of severity.

My objective is to train a model for binary classification to predict whether a patient has a heart disease or not. In order to convert the target into a binary column, the values that indicate the presence of disease are all turned into 1.

In [389]:
df.num.loc[df.num != 0] = 1

/var/folders/lf/3_22z6ms1hscctw3lvvwn5q80000gn/T/ipykernel_4032/4041446238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.num.loc[df.num != 0] = 1


And I change the target column name to something more descriptive.

In [390]:
df["disease"] = df.num
df.drop("num", axis=1, inplace=True)
df.disease.value_counts()

1    509
0    411
Name: disease, dtype: int64

The dataset is balanced!

## Setting up the validation framework

In [391]:
from sklearn.model_selection import train_test_split

In [392]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [393]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [394]:
y_train = df_train.disease.values
y_val = df_val.disease.values
y_test = df_test.disease.values

In [395]:
df_train.drop("disease", axis=1, inplace=True)
df_val.drop("disease", axis=1, inplace=True)
df_test.drop("disease", axis=1, inplace=True)

## Feature importance analysis

### Feature importance of categorical columns: mutual information score

In [396]:
from sklearn.metrics import mutual_info_score

In [397]:
categorical = ["sex", "dataset", "cp", "fbs", "restecg", "exang", "slope"]

In [398]:
def mutual_info_disease_score(series):
    return mutual_info_score(series, df_full_train.disease)

In [399]:
mi = df_full_train[categorical].apply(mutual_info_disease_score)
mi.sort_values(ascending=False)

cp         0.147004
exang      0.097273
dataset    0.091677
sex        0.060734
slope      0.022916
restecg    0.007835
fbs        0.005028
dtype: float64

### Feature importance of numerical columns: correlation

In [400]:
numerical = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']

In [401]:
df_full_train[numerical].corrwith(df_full_train.disease).abs().sort_values(ascending=False)

thalch      0.385894
oldpeak     0.369913
age         0.256861
chol        0.236068
trestbps    0.099594
dtype: float64

## Model selection

To select the best model for this classification task,
I first train the following models with their default settings and compare their ROC AUC scores:
- Logistic Regression
- Decision Tree
- Random Forest
- XG Boost

In [402]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.metrics import roc_auc_score

The training and validation dataframes are vectorized in order to encode the categorical features.

In [403]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')
val_dict = df_val[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

In [404]:
log_reg = LogisticRegression(max_iter=1500)
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(random_state=1)

In [405]:
log_reg.fit(X_train, y_train)
dt.fit(X_train, y_train)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [406]:
y_pred_log_reg = log_reg.predict_proba(X_val)[:, 1]
y_pred_dt = dt.predict_proba(X_val)[:, 1]
y_pred_rf = rf.predict_proba(X_val)[:, 1]
auc_log_reg = roc_auc_score(y_val, y_pred_log_reg)
auc_dt = roc_auc_score(y_val, y_pred_dt)
auc_rf = roc_auc_score(y_val, y_pred_rf)

In [407]:
print("logistic regression roc_auc:", auc_log_reg)
print("decision tree roc_auc:", auc_dt)
print("random forest roc_auc:", auc_rf)

logistic regression roc_auc: 0.9147619047619047
decision tree roc_auc: 0.726190476190476
random forest roc_auc: 0.8972619047619048


In [408]:
features = dv.get_feature_names_out()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [409]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model_xgb = xgb.train(xgb_params, dtrain, num_boost_round=10)
y_pred_xgb = model_xgb.predict(dval)
auc_xgb = roc_auc_score(y_val, y_pred_xgb)
print("XGBoost roc_auc:", auc_xgb)

XGBoost roc_auc: 0.8803571428571428


## Model fine-tuning

Logistic regression has the best ROC AUC score,
so I experiment with different parameters to see if the result can be improved. The default solver is changed to `'liblinear'` to be able to test `'l1'` and `'l2'` penalties.

The parameters tested are regularization and penalty.

In [410]:
scores = []

for c in [0.001, 0.01, 0.1, 0.5, 1, 5, 10]:
    for p in ['l1', 'l2']:
        log_reg = LogisticRegression(C=c, solver='liblinear', penalty=p, max_iter=1500)
        log_reg.fit(X_train, y_train)

        y_pred = log_reg.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        scores.append((c, p, auc))
        
columns = ['regularization', 'penalty', 'auc']
df_scores = pd.DataFrame(scores, columns=columns)
df_scores

,regularization,penalty,auc
0,0.001,l1,0.699643
1,0.001,l2,0.827143
2,0.010,l1,0.797024
3,0.010,l2,0.886786
4,0.100,l1,0.899524
5,0.100,l2,0.911071
6,0.500,l1,0.919524
7,0.500,l2,0.915000
8,1.000,l1,0.918333
9,1.000,l2,0.915000


The result is slightly improved by using `'l1'` penalty with the default regularization of `C=1.0`.

I now train this model with the full training dataset (df_train + df_val) and test it with the held out data (df_test).

In [415]:
df_full_train = df_full_train.reset_index(drop=True)
y_train = df_full_train.disease.values

train_dict = df_full_train[categorical + numerical].to_dict(orient='records')
test_dict = df_test[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dict)
X_test = dv.transform(test_dict)

log_reg = LogisticRegression(max_iter=1500, solver='liblinear', penalty='l1')
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)

print("logistic regression roc_auc:", auc)

logistic regression roc_auc: 0.8988379204892967


I also test a `'lbfgs'`
 solver with both allowed penalties to see if it offers a better result:

In [416]:
scores = []
for p in ['none', 'l2']:
    log_reg = LogisticRegression(max_iter=1500, solver='lbfgs', penalty=p)
    log_reg.fit(X_train, y_train)

    y_pred = log_reg.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)
    
    scores.append((p, auc))

columns = ['penalty', 'auc']
df_scores = pd.DataFrame(scores, columns=columns)
df_scores

,penalty,auc
0,none,0.894679
1,l2,0.895902


The best results seem to come from a Logistic Regression model with a `'liblinear'` solver and `L1` penalty.
I now train the model with the full training set and test it with the hold out test data.

In [413]:
df_full_train = df_full_train.reset_index(drop=True)
y_train = df_full_train.disease.values

train_dict = df_full_train[categorical + numerical].to_dict(orient='records')
test_dict = df_test[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dict)
X_test = dv.transform(test_dict)

log_reg = LogisticRegression(max_iter=1500, solver='liblinear', penalty='l1')
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)

print("logistic regression roc_auc:", auc)

logistic regression roc_auc: 0.8988379204892967


The model has a good result with unseen testing data.

## Saving the best model

I will use BentoML to save and test the model API locally and then containerize and deploy it to AWS Elastic Container Service.

In [224]:
import bentoml

In [380]:
bentoml.sklearn.save_model('heart_disease_model', log_reg, custom_objects={"dictVectorizer" : dv})

Model(tag="heart_disease_model:7apds5s6s25s3ahg", path="/Users/nineve/bentoml/models/heart_disease_model/7apds5s6s25s3ahg/")

The code to train and save the model is converted to a Python file `train.py` and then cleaned up.

In [382]:
# !jupyter nbconvert --to script notebook.ipynb

[NbConvertApp] Converting notebook notebook.ipynb to script
[NbConvertApp] Writing 11325 bytes to notebook.py


In [250]:
import requests

data={
    "sex": "Female",
    "dataset": "Hungary",
    "cp": "atypical angina",
    "fbs": "False",
    "restecg": "st-t abnormality",
    "exang": "False",
    "slope": "flat",
    "age": 31,
    "trestbps": 100.0,
    "chol": 219.0,
    "thalch": 150.0,
    "oldpeak": 0.0
}

requests.post(
   "http://localhost:3000/classify",
   headers={"content-type": "application/json"},
   json=data,
).json()

{'status': 'Normal heart'}

In [ ]:
!jupyter nbconvert --to script notebook.ipynb